# Predicting Airbnb Listing Prices with Multiple Regression

- Author: Sebastian Schuetz, swschuetz@outlook.com
- Date: Feb 12, 2025

In this exercise, we will build a multiple regression model to predict Airbnb listing prices. We will work through the following steps:

1. **Data Loading & Exploratory Data Analysis (EDA)**
2. **Splitting the Data into Training and Testing Sets**
3. **Fitting Regression Models**
4. **Evaluating Model Performance on the Test Set**

## Step 1. Data Loading

You are provided with a CSV file named airbnb_listings.csv containing the following variables:
- **Price**: Listing price per night in USD (Dependent Variable)
- **Minimum_Nights**: Minimum number of nights required to book
- **Accommodates**: Number of guests the listing can accommodate
- **Bedrooms**: Number of bedrooms in the listing
- **Bathrooms**: Number of bathrooms in the listing
- **Distance_CityCenter**: Distance from the listing to the city center (in miles)
- **Review_Score**: Overall review score (on a scale from 1 to 5)
- **Number_of_Reviews**: Total number of reviews the listing has received
- **Amenities_Score**: A score (1–10) representing the quantity and quality of amenities offered
- **Superhost**: Indicator variable where 1 = Host is a Superhost, 0 = otherwise
- **Listing_Age**: Number of years since the listing was first posted
- **Room_Type**: Category of the listing (e.g., “Entire home/apt”, “Private room”, “Shared room”)

In this step, we generate the data.

In [ ]:
import numpy as np
import pandas as pd

# Set a seed for reproducibility
np.random.seed(42)

# Number of listings to simulate
n = 100

# Generate predictor variables
Minimum_Nights = np.random.randint(1, 11, n)  # between 1 and 10 nights
Accommodates = np.random.randint(1, 11, n)      # can accommodate 1 to 10 guests
Bedrooms = np.random.randint(0, 6, n)           # between 0 and 5 bedrooms
Bathrooms = np.round(np.random.uniform(1, 3, n), 1)  # between 1.0 and 3.0 bathrooms
Distance_CityCenter = np.round(np.random.uniform(0.5, 10, n), 1)  # between 0.5 and 10 miles
Review_Score = np.round(np.random.uniform(3.0, 5.0, n), 1)  # review scores between 3.0 and 5.0
Number_of_Reviews = np.random.randint(0, 501, n)  # 0 to 500 reviews
Amenities_Score = np.random.randint(1, 11, n)       # score between 1 and 10
Superhost = np.random.choice([0, 1], n)             # 0 (not superhost) or 1 (superhost)
Listing_Age = np.random.randint(0, 11, n)           # listing age between 0 and 10 years

# Generate categorical variable 'Room_Type'
room_types = ["Entire home/apt", "Private room", "Shared room"]
# Assume probabilities (for example, 50% Entire home/apt, 40% Private room, 10% Shared room)
Room_Type = np.random.choice(room_types, n, p=[0.5, 0.4, 0.1])

# Simulate Price using a synthetic linear relationship with noise.
# You can adjust the coefficients to suit your simulation needs.
base_price = 50
Price = (
    base_price +
    Accommodates * 5 +
    Bedrooms * 20 +
    Bathrooms * 15 -
    Distance_CityCenter * 3 +
    Review_Score * 10 +
    Amenities_Score * 5 +
    Superhost * 10 -
    Listing_Age * 2 +
    np.random.normal(0, 10, n)  # error term with mean 0 and standard deviation 10
)
Price = np.round(Price, 2)  # round to 2 decimal places

# Create a DataFrame with all the generated data
data = pd.DataFrame({
    'Price': Price,
    'Minimum_Nights': Minimum_Nights,
    'Accommodates': Accommodates,
    'Bedrooms': Bedrooms,
    'Bathrooms': Bathrooms,
    'Distance_CityCenter': Distance_CityCenter,
    'Review_Score': Review_Score,
    'Number_of_Reviews': Number_of_Reviews,
    'Amenities_Score': Amenities_Score,
    'Superhost': Superhost,
    'Listing_Age': Listing_Age,
    'Room_Type': Room_Type
})

## Step 2. Splitting the Data into Training and Testing Sets

We split our data so that we can train the model on one set and evaluate its performance on unseen data.

<div style="border: 2px solid red; padding: 10px; color: red;">
<b>Experiment 2:</b> Change the data split by reducing the test size to 0.05 (i.e., test_size=0.05). How does this affect the MSE for all models?
</div>

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data: 70% for training, 30% for testing
train_data, test_data = train_test_split(data, test_size=0.20, random_state=42)

print("Full data set shape", data.shape,"\n---")
print("Training set shape:", train_data.shape)
print("Testing set shape:", test_data.shape)

## 3. Fitting a Model

### 3.1 Fitting a Simple Model (Too Simple, Underfitting)

We start with specifying a simple model that uses the number of *bedrooms* to predict the nightly price.

$$
\begin{aligned}
\text{Price}_i &= \beta_0 + \beta_1 \cdot \text{Accomodates} + \epsilon_i
\end{aligned}
$$

Here, we’ll use the Ordinary Least Squares (OLS) method to estimate the regression weights ($\beta_i$).

<div style="border: 2px solid red; padding: 10px; color: red;">
<b>Experiment 1:</b> Change the predictor variable from <i>Accomodates</i> to <i>Bedrooms</i> by uncommenting the code below the comment on Configuration 2. Explore: How does this affect the variance explained (i.e., R^2) and MSE?
</div>

In [ ]:
import statsmodels.formula.api as smf

# Configuration 1: Define the model formula for predicing price based on accomodates
formula = ("Price ~ Accommodates")

# Configuration 2: Define the model formula for predicing price based on bedrooms. To use this configuration, 
# uncomment the next line (i.e., by removing the leading #)
# formula = ("Price ~ Bedrooms")

# Fit the model using OLS and the training_data
simple_model = smf.ols(formula=formula, data=train_data).fit()

# Show model summary
simple_model.summary()

### Simple Model: Predictive Power on Trained Data
Let's examine the actual vs. the predicted prices based on this simple model.

In [ ]:
import matplotlib.pyplot as plt

# Extract actual and predicted values
actual = train_data["Price"]
predicted = simple_model.fittedvalues

# Create scatter plot
plt.figure(figsize=(8,6))
plt.scatter(actual, predicted, alpha=0.5, label="Predicted vs Actual")

# Add perfect prediction reference line
plt.plot([actual.min(), actual.max()], [actual.min(), actual.max()], 'r--', label="Perfect Fit")

# Labels and title
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.title("Predicted vs Actual Price (Simple Model)")
plt.legend()
plt.grid(True)

# Show plot
plt.show()

## 3.2 Fitting a Multiple Regression Model (Full model, good fit)

Now, we fit a moderate model using all the main predictors, including a categorical indicator for Room_Type. The mathematical specification of the model is:

$$
\begin{aligned}
\text{Price}_i &= \beta_0 + \beta_1 \cdot \text{Minimum\_Nights}_i + \beta_2 \cdot \text{Accommodates}_i + \beta_3 \cdot \text{Bedrooms}_i \\
&\quad + \beta_4 \cdot \text{Bathrooms}_i + \beta_5 \cdot \text{Distance\_CityCenter}_i + \beta_6 \cdot \text{Review\_Score}_i \\
&\quad + \beta_7 \cdot \text{Number\_of\_Reviews}_i + \beta_8 \cdot \text{Amenities\_Score}_i + \beta_9 \cdot \text{Superhost}_i \\
&\quad + \beta_{10} \cdot \text{Listing\_Age}_i + \beta_{11} \cdot \text{Room\_Type\_Private}_i \\
&\quad + \beta_{12} \cdot \text{Room\_Type\_Shared}_i + \epsilon_i
\end{aligned}
$$


Will use OLS again to estimate the regression weights ($\beta_i$).

In [ ]:
import statsmodels.formula.api as smf

# Define the model formula:
# "Price" is our dependent variable.
# The tilde (~) separates the dependent variable from the predictors.
# C(Room_Type) tells statsmodels to treat Room_Type as a categorical variable.
formula = ("Price ~ Minimum_Nights + Accommodates + Bedrooms + Bathrooms + "
           "Distance_CityCenter + Review_Score + Number_of_Reviews + Amenities_Score + "
           "Superhost + Listing_Age + C(Room_Type)")

#formula = ("Price ~ Minimum_Nights")

# Fit the model using the training data
full_model = smf.ols(formula=formula, data=train_data).fit()

# Print the model summary
full_model.summary()

### Full Model: Predictive Power on Trained Data

In [ ]:
import matplotlib.pyplot as plt

# Extract actual and predicted values
actual = train_data["Price"]
predicted = full_model.fittedvalues

# Create scatter plot
plt.figure(figsize=(8,6))
plt.scatter(actual, predicted, alpha=0.5, label="Predicted vs Actual")

# Add perfect prediction reference line
plt.plot([actual.min(), actual.max()], [actual.min(), actual.max()], 'r--', label="Perfect Fit")

# Labels and title
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.title("Predicted vs Actual Price (Full Model)")
plt.legend()
plt.grid(True)

# Show plot
plt.show()

In [ ]:
# Create polynomial terms for two predictors in the dataset
train_data['Accommodates_sq'] = train_data['Accommodates'] ** 2
train_data['Distance_sq'] = train_data['Distance_CityCenter'] ** 2

test_data['Accommodates_sq'] = test_data['Accommodates'] ** 2
test_data['Distance_sq'] = test_data['Distance_CityCenter'] ** 2

# Overfitting model: include polynomial terms and interaction between Review_Score and Number_of_Reviews
formula_over = (
    "Price ~ Minimum_Nights + Accommodates + Accommodates_sq + Bedrooms + Bathrooms + "
    "Distance_CityCenter + Distance_sq + Review_Score * Number_of_Reviews + "
    "Amenities_Score + Superhost + Listing_Age + C(Room_Type)"
)
complex_model = smf.ols(formula=formula_over, data=train_data).fit()
complex_model.summary()

## 4. Model Evaluation

We now evaluate all three models using both training and test data. 

The goal is to see:
- Underfitting: High error on both training and test sets.
- Baseline Model: A balanced fit with moderate error on both sets.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

def evaluate_model(model, train, test, model_name="Model"):
    # Predict on training and test sets
    predictions_train = model.predict(train)
    predictions_test = model.predict(test)
    
    # Calculate error metrics
    mse_train = mean_squared_error(train['Price'], predictions_train)
    rmse_train = np.sqrt(mse_train)
    mae_train = mean_absolute_error(train['Price'], predictions_train)
    
    mse_test = mean_squared_error(test['Price'], predictions_test)
    rmse_test = np.sqrt(mse_test)
    mae_test = mean_absolute_error(test['Price'], predictions_test)
    
    print(f"\n{model_name} Performance:")
    print(f"Training - MSE: {mse_train:.2f}, RMSE: {rmse_train:.2f}, MAE: {mae_train:.2f}")
    print(f"Testing  - MSE: {mse_test:.2f}, RMSE: {rmse_test:.2f}, MAE: {mae_test:.2f}")

# Prepare datasets for evaluation
# For the baseline model, no additional columns are needed.
# For the overfitting model, ensure that polynomial terms are present in test_data as well.
train_eval = train_data.copy()
test_eval = test_data.copy()

# Evaluate all three models:
evaluate_model(simple_model, train_eval, test_eval, model_name="Simple Model (Underfitting)")
evaluate_model(full_model, train_eval, test_eval, model_name="Full Model")